<h1>ZMGSN Lista 5. - Architektura Sieci typu Transformer</h1>

<h2>Opis zadania</h2>

W ramach zadania należy:
<ol>
<li>Zapoznać się z poniższym eksperymentem z wykorzystaniem sieci typu Transformer</li>
<li>Dokonać analizy wpływu hiperparametrów eksperymentu, np. kroku uczenia, rozmiaru pakietu (ang. <i>batch size</i>), liczby epok na wyniki sieci typu Transformer</li>
<li>Wykorzystać inne architektury wstępnie wyuczonych sieci typu Transformer (np. RoBERTa, XLM-RoBERTa, DistilBERT, AlBERT, DeBERTa, XLNet, MPNet, LaBSE itp.) w celu uzasadnienia w jaki sposób ich architektura, sposób uczenia i zbiór uczący mogły mieć wpływ na wyniki</li>
<li>Wykorzystać inne rozmiary wstępnie wyuczonych sieci typu Transformer (np. RoBERTa-Large, XLM-RoBERTa-XL itp.) w celu uzasadnienia w jaki sposób ich architektura, sposób uczenia i zbiór uczący mogły mieć wpływ na wyniki</li>
<li>Dokonać modyfikacji rozszerzenia wstępnie wyuczonej sieci typu Transformer w celu zbadania wpływu architektury na wyniki</li>
<li>Zaimplementować własne warianty rozszerzeń architektury wstępnie wyuczonej sieci typu Transformer w celu zbadania ich wpływu na wyniki</li>
<li>Zbadać wpływ maksymalnej długości tekstu oraz różnych strategii paddingu dla każdego z wykorzystanych wstępnie wyuczonych modeli</li>
<li>Dokonać ewaluacji różnych wstępnie wyuczonych modeli sieci typu Transformer (różne rozmiary modeli oraz typy), modyfikacji oryginalnego rozszerzenia oraz opracowanych rozszerzeń zgodnie z punktami 2., 5., 6. i 7.</li>
<li>Dokonać usystematyzowanej ewaluacji porównawczej wszystkich modeli sieci typu transformer, ich wariantów, wykorzystanych rozszerzeń w celu zbadania różnic, podobieństw oraz analogicznych cech ich charakterystyki.</li>
<li>Opracować procedurę ewaluacji jakości działania modeli sieci typu Transformer, uwzględniającą różne metody wizualizacji (np. wykresy, miary, klasy), klasteryzacji, redukcji wymiarów (np. t-SNE), walidacji krzyżowej, wpływu charakterystyki zbioru uczącego na działanie modelu, podatności na semantykę tekstów w zbiorze uczącym i testowym itp.</li>
</ol>

Ocenie podlegać będzie jakość wykonania zadania, w tym:
<ol>
<li>Właściwe wykonanie zadań</li>
<li>Rzetelne opracowanie wyników, uwzględniające analizę jakościową i ilościową</li>
<li>Opracowanie wniosków mających na celu wyjaśnienie badanych zjawisk i uzyskanych wyników</li>
<li>Opracowanie i wyjaśnienie kodu źródłowego</li>
</ol>


<h2>Import używanych bibliotek</h2>

In [ ]:
import os

import gdown
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, AutoModel, AutoTokenizer

<h2>Inicjalizacja ziarna generatora liczb pseudolosowych</h2>

In [ ]:
torch.manual_seed(0)

<h2>Określenie domyślnego urządzenia na podstawie sprawdzenia dostępności karty graficznej</h2>

In [ ]:
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

Using device cuda:0


<h2>Pobranie i rozpakowanie zbioru danych</h2>

In [ ]:
if os.path.exists('data.csv'):
  os.remove('data.csv')

In [ ]:
url = 'https://drive.google.com/uc?id=1HSnB-D0dKDI2bE9iOsp-Vr8tumihdvbH'
output = 'data.csv'

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1HSnB-D0dKDI2bE9iOsp-Vr8tumihdvbH
To: /content/data.csv
100%|██████████| 467k/467k [00:00<00:00, 111MB/s]


'data.csv'

<h2>Wczytanie zbioru danych</h2>

In [ ]:
df = pd.read_csv('data.csv')
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


<h2>Podział zbioru na podzbiór uczący i testowy</h2>

In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df['label'],
                                                                    random_state=2018,
                                                                    test_size=0.3,
                                                                    stratify=df['label'])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
                                                                random_state=2018,
                                                                test_size=0.5,
                                                                stratify=temp_labels)

<h2>Pobranie modelu wstępnie wyuczonej sieci typu Transformer</h2>

In [ ]:
# Pobranie oraz wczytanie modelu transformera
pretrained_model = AutoModel.from_pretrained('bert-base-uncased')

# Pobranie oraz wczytanie dedykowanego tokenizatora
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

<h2>Tokenizacja oraz wygenerowanie wektorowych reprezentacji tekstów</h2>

In [ ]:
# Tokenizacja i wygenerowanie reprezentacji wektorowych tekstów ze zbioru uczącego
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = 25,
    padding='max_length',
    truncation=True
)

# Tokenizacja i wygenerowanie reprezentacji wektorowych tekstów ze zbioru walidacyjnego
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 25,
    padding='max_length',
    truncation=True
)

# Tokenizacja i wygenerowanie reprezentacji wektorowych tekstów ze zbioru testowego
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 25,
    padding='max_length',
    truncation=True
)

<h2>Konwersja list na tensory</h2>

In [ ]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

<h2>Przygotowanie instancji klas typu DataLoader</h2>

In [ ]:
# Określenie rozmiaru pakietu (ang. batch size)
batch_size = 32

# Utworzenie obiektu klasy nadrzędnej dla zbiorów: uczącego, walidacyjnego i teestowego
train_data = TensorDataset(train_seq, train_mask, train_y)

# Przygotowanie obiektu klasy pozwalającej na próbkowanie zbioru uczącego
train_sampler = RandomSampler(train_data)

# Przygotowanie obhiektu klasy DataLoader dla zbioru uczącego
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Przygotowanie klasy nadrzędnej dla zbioru walidacyjnego
val_data = TensorDataset(val_seq, val_mask, val_y)

# Przygotowanie obiektu klasy pozwalającej na próbkowanie zbioru uczącego
val_sampler = SequentialSampler(val_data)

# Przygotowanie obhiektu klasy DataLoader dla zbioru walidacyjnego
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

<h2>Przygotowanie rozszerzenia architektury wstępnie wyuczonego modelu sieci typu Transformer</h2>

In [ ]:
# Zamrozenie wszystkich parametrów pierwotnej sieci
for param in pretrained_model.parameters():
    param.requires_grad = False

In [ ]:
class Ext_Arch(nn.Module):

    def __init__(self, pretrained_model):
        super(Ext_Arch, self).__init__()

        self.pretrained_model = pretrained_model

        self.dropout = nn.Dropout(0.1)

        self.relu =  nn.ReLU()

        self.fc1 = nn.Linear(768,512)

        self.fc2 = nn.Linear(512,2)

        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sent_id, mask):

        _, cls_hs = self.pretrained_model(sent_id, attention_mask=mask, return_dict=False)

        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        x = self.fc2(x)

        x = self.softmax(x)

        return x

<h2>Konfiguracja eksperymentu</h2>

In [ ]:
# Inicjalizacja rozszerzonej architektury pierwotnym modelem
model = Ext_Arch(pretrained_model)

# Przeniesienie modelu do pamięci domyślnego urządzenia
model = model.to(device)

In [ ]:
# Inicjalizacja optymalizatora
optimizer = AdamW(model.parameters(),lr = 1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Obliczenie wag klas
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)

print("Class Weights:", class_weights)

Class Weights: [0.57743559 3.72848948]


In [ ]:
# Konwersja listy z wagami klas do typu tensorowego
weights= torch.tensor(class_weights,dtype=torch.float)

# Przeniesienie wag do pamięci domyślnego urządzenia
weights = weights.to(device)

# Określenie funkcji straty
cross_entropy  = nn.NLLLoss(weight=weights)

# Określenie liczby epok
epochs = 10

<h2>Kalibracja modelu (ang. fine-tuning)</h2>

In [ ]:
def train():

    model.train()
    total_loss, total_accuracy = 0, 0

    # przygotowanie listy do przechowywania predykcji modelu
    total_preds=[]

    for step,batch in enumerate(train_dataloader):

        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch

        model.zero_grad()

        preds = model(sent_id, mask)

        loss = cross_entropy(preds, labels)

        total_loss = total_loss + loss.item()

        loss.backward()

        # Normalizacja wartości gradientów
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        preds=preds.detach().cpu().numpy()

    total_preds.append(preds)

    avg_loss = total_loss / len(train_dataloader)

    # Predykcje modelu mają wymiary (liczba pakietów, rozmiar pakietu, liczba klas).
    # Przekształcenie ich do wymiarów (liczba próbek, liczba klas)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [ ]:
def evaluate():

    print("\nEvaluating...")

    model.eval()

    total_loss, total_accuracy = 0, 0

    total_preds = []

    for step,batch in enumerate(val_dataloader):

        if step % 50 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)

            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        with torch.no_grad():

            preds = model(sent_id, mask)

            loss = cross_entropy(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

    avg_loss = total_loss / len(val_dataloader)

    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [ ]:
# Inicjalizacja początkowej wartości funkcji straty
best_valid_loss = float('inf')

# Inicjalizacja list na wartości funkcji straty na zbiorze uczącym i walidacyjnym
train_losses=[]
valid_losses=[]

for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, _ = train()

    valid_loss, _ = evaluate()

    # zapisanie najlepszego modelu
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
  Batch    50  of    122.
  Batch   100  of    122.

Evaluating...

Training Loss: 0.673
Validation Loss: 0.649

 Epoch 2 / 10
  Batch    50  of    122.
  Batch   100  of    122.

Evaluating...

Training Loss: 0.641
Validation Loss: 0.623

 Epoch 3 / 10
  Batch    50  of    122.
  Batch   100  of    122.

Evaluating...

Training Loss: 0.619
Validation Loss: 0.595

 Epoch 4 / 10
  Batch    50  of    122.
  Batch   100  of    122.

Evaluating...

Training Loss: 0.594
Validation Loss: 0.568

 Epoch 5 / 10
  Batch    50  of    122.
  Batch   100  of    122.

Evaluating...

Training Loss: 0.570
Validation Loss: 0.544

 Epoch 6 / 10
  Batch    50  of    122.
  Batch   100  of    122.

Evaluating...

Training Loss: 0.550
Validation Loss: 0.533

 Epoch 7 / 10
  Batch    50  of    122.
  Batch   100  of    122.

Evaluating...

Training Loss: 0.527
Validation Loss: 0.499

 Epoch 8 / 10
  Batch    50  of    122.
  Batch   100  of    122.

Evaluating...

Training Loss: 0.508
Validat

In [ ]:
# Wczytanie wartości parametrów najlepszego modelu
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

<h2>Wygenerowanie predykcji za pomocą skalibrowanego modelu oraz ocena ich jakości</h2>

In [ ]:
# Wygenerowanie predykcji dla zbioru testowego
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()

In [ ]:
# Ocena jakości predykcji modelu
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.98      0.79      0.88       724
           1       0.40      0.88      0.54       112

    accuracy                           0.80       836
   macro avg       0.69      0.83      0.71       836
weighted avg       0.90      0.80      0.83       836

